## Tiff visualizer

### Metadati immagine

In [4]:
import tifffile as tiff
import matplotlib.pyplot as plt
import os

def mostra_metadati_tiff(file_path):
    """
    Mostra i metadati di un file TIFF.
    :param file_path: Percorso del file TIFF.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"Il file {file_path} non esiste.")
    
    with tiff.TiffFile(file_path) as tif:
        # Stampa i metadati della descrizione (se presenti)
        if tif.pages[0].description:
            try:
                metadati = eval(tif.pages[0].description)
                print("Metadati del file TIFF:")
                for chiave, valore in metadati.items():
                    print(f"{chiave}: {valore}")
            except:
                print("Attenzione: i metadati della descrizione non sono in un formato valido.")
        else:
            print("Nessun metadato trovato nella descrizione del file TIFF.")
        
        # Stampa i metadati dei tag TIFF
        print("\nMetadati dei tag TIFF:")
        for tag in tif.pages[0].tags.values():
            print(f"{tag.name}: {tag.value}")

def load_and_display_tiff(file_path, load=False, show=False):
    """
    Processa un file TIFF, mostrando metadati e, opzionalmente, caricando e visualizzando l'immagine.
    :param file_path: Percorso del file TIFF.
    :param load: Se True, carica l'immagine in memoria.
    :param show: Se True, visualizza i canali dell'immagine.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"Il file {file_path} non esiste.")
    
    with tiff.TiffFile(file_path) as tif:
        # Mostra i metadati
        mostra_metadati_tiff(file_path)
        
        # Informazioni sulla forma dell'immagine
        image_shape = tif.pages[0].shape
        if len(image_shape) == 3:
            height, width, channels = image_shape
            print(f"\nL'immagine è multicanale con dimensioni: {height}x{width}x{channels}")
        else:
            height, width = image_shape
            print(f"\nL'immagine è single-channel con dimensioni: {height}x{width}")

    if load:
        # Carica l'immagine
        image = tiff.imread(file_path)
        print(f"\nImmagine caricata con successo: {image.shape}")
        
        # Seleziona i canali specifici per RGB, Red Edge e NIR
        selected_channels = [0, 1, 2, 3, 7]  # Indici delle bande (0-based: B2, B3, B4, B5, B8)
        image = image[:, :, selected_channels]  # Seleziona i canali

        if show:
            # Nomi delle bande selezionate
            band_names = ['Blue (B2)', 'Green (B3)', 'Red (B4)', 'Red Edge (B5)', 'NIR (B8)']
            
            # Visualizza ogni canale
            for i, band in enumerate(band_names):
                plt.figure(figsize=(6, 6))
                plt.imshow(image[:, :, i], cmap='gray')
                plt.title(f'Canale {i+1}: {band}')
                plt.colorbar()
                plt.show()

# Esempio di utilizzo
file_path = '/home/brus/Projects/wavelet/images/ortho/190529_pi_5a_5b_re.tif'
load_and_display_tiff(file_path)

Attenzione: i metadati della descrizione non sono in un formato valido.

Metadati dei tag TIFF:
ImageWidth: 29159
ImageLength: 27051
BitsPerSample: (16, 16, 16, 16, 16)
Compression: 5
PhotometricInterpretation: 1
ImageDescription: 5a, 5b
SamplesPerPixel: 5
PlanarConfiguration: 1
Software: Poplar Island
DateTime: 2019-05-29
TileWidth: 256
TileLength: 256
TileOffsets: (51, 1501, 2951, 4401, 5851, 7301, 8751, 10201, 11651, 184352, 504013, 511103, 512553, 514003, 515453, 516903, 130466755, 130468205, 130469655, 130471105, 130472555, 130474005, 130475455, 130476905, 130478355, 130479805, 130481255, 130482705, 130484155, 130485605, 130487055, 130488505, 233016405, 233017855, 233019305, 233020755, 233022205, 233023655, 233025105, 233026555, 233028005, 233029455, 233030905, 233032355, 233033805, 233035255, 233036705, 233038155, 245346095, 245347545, 245348995, 245350445, 245351895, 245353345, 245354795, 245356245, 245357695, 245359145, 245360595, 245362045, 245363495, 245364945, 245366395, 245

## 5 channel selector

In [ ]:
import tifffile as tiff
import matplotlib.pyplot as plt
import os
import numpy as np

# Bande selezionate: RGB (B2, B3, B4), Red Edge (B5), NIR (B8)
SELECTED_CHANNELS = [1, 2, 3, 4, 8]  # Indici delle bande da 1 a 10
BAND_NAMES = ['Blue (B2)', 'Green (B3)', 'Red (B4)', 'Red Edge (B5)', 'NIR (B8)']

def load_and_display_tiff(file_path, output_path):
    """
    Legge un file TIFF multicanale, seleziona le bande specificate e salva il nuovo file.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"Il file {file_path} non esiste.")
    
    image = tiff.imread(file_path)
    if image.ndim == 3 and image.shape[2] >= max(SELECTED_CHANNELS):
        selected_image = image[:, :, [c - 1 for c in SELECTED_CHANNELS]]  # Indici Python (0-based)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        tiff.imwrite(output_path, selected_image)
        print(f"File salvato: {output_path}")
    else:
        print(f"File {file_path} non contiene abbastanza canali.")

def explore_and_process(root_dir, output_dir):
    """
    Esplora le sottocartelle e processa tutti i file TIFF, salvando i risultati in nuove cartelle.
    """
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(".tif") or file.lower().endswith(".tiff"):
                input_path = os.path.join(subdir, file)
                relative_path = os.path.relpath(subdir, root_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                output_path = os.path.join(output_subdir, file)
                load_and_display_tiff(input_path, output_path)

# Esempio di utilizzo
root_directory = '/home/brus/Projects/wavelet/examples/EuroSAT_MS'
output_directory = '/home/brus/Projects/wavelet/EuroSAT_MS_5bands'
explore_and_process(root_directory, output_directory)

File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_215.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_1305.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_1264.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_1613.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_1300.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_2294.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_1289.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_1108.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_2023.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_2017.tif
File salvato: /home/brus/Projects/wavelet/EuroSAT_MS_5bands/Industrial/Industrial_2061.tif


## Channels devider

In [ ]:
import os
import tifffile as tiff
import numpy as np

class ImageProcessor:
    def __init__(self, file_path, window_size=64):
        self.file_path = file_path
        self.window_size = window_size
        self.image = None
        self.height = None
        self.width = None
        self.channels = None
        self.output_prefix = os.path.splitext(os.path.basename(file_path))[0]
        self.output_base_path = os.path.dirname(file_path)

    def load_image(self):
        with tiff.TiffFile(self.file_path) as tif:
            self.image = tif.asarray()
            if len(self.image.shape) == 3:
                self.height, self.width, self.channels = self.image.shape
                print(f"L'immagine è multicanale con dimensioni: {self.height}x{self.width}x{self.channels}")
            else:
                self.height, self.width = self.image.shape
                self.channels = 1
                print(f"L'immagine è single-channel con dimensioni: {self.height}x{self.width}")

    def save_channel(self, channel_index):
        channel_folder = os.path.join(self.output_base_path, f'{self.output_prefix}_channel_{channel_index + 1}')
        os.makedirs(channel_folder, exist_ok=True)

        window_count = 0
        for y in range(0, self.height, self.window_size):
            y_end = min(y + self.window_size, self.height)
            for x in range(0, self.width, self.window_size):
                x_end = min(x + self.window_size, self.width)

                if self.channels > 1:
                    window = self.image[y:y_end, x:x_end, channel_index]
                else:
                    window = self.image[y:y_end, x:x_end]

                center_x = x + (x_end - x) // 2
                center_y = y + (y_end - y) // 2

                output_filename = f'{self.output_prefix}_channel_{channel_index + 1}_{window_count}.tif'
                output_path = os.path.join(channel_folder, output_filename)

                metadata = {
                    'SourceFile': self.file_path,
                    'Channel': channel_index + 1,
                    'CenterX': center_x,
                    'CenterY': center_y,
                }

                tiff.imwrite(output_path, window, description=str(metadata))
                window_count += 1

                if window_count % 30000 == 0:
                    print(f'Processed {window_count} windows for channel {channel_index + 1}')
                    

    def process_image(self):
        self.load_image()
        for i in range(self.channels):
            self.save_channel(i)
        print(f'Successfully saved channels as separate folders of TIFF images.')

# Esempio di utilizzo
file_path = "/home/brus/Projects/wavelet/images/ortho/190529_pi_5ab_re.tif"  # Sostituisci con il percorso del tuo file TIFF
processor = ImageProcessor(file_path)
processor.process_image()